# Dynamic function comparisons between the paper and code implementation (specfically `get_stationary_states_arange_x_homogenousNetwork.py`)

To discretize the dynamic function updates, clever methods were utilized to update the variables using the Euler scheme. However, for compurational efficiency certain variables would be factored out of the code. 
Further, certain variables were scaled to speed up the simulation process. The combination is thus confusion for those who are not engaged with the code on a regular basis. 
Thus, this notebook aims to clarify some of the differences and how each equations are implemented in the code.

Specifically, this notebook focuses on the model `get_stationary_states_arange_x_homogenousNetwork.py`, other models or scripts should be able to be extrapolated from the analysis on this model.


## Equation 5:

### Paper's equation:
![Alt text](image.png) 

$$
\begin{align*}
C_i \frac{dV_i}{dt} & = g_{leak} (V_{rest} - V_i) + g_{syn, i}(t)(V_{syn} - V_i) + I_{stim}(t) + I_{noise, i}(t) \\
\implies dV_i & = \frac{dt}{C_i} \Big(  g_{leak} (V_{rest} - V_i) + g_{syn, i}(t)(V_{syn} - V_i) + I_{stim}(t) + I_{noise, i}(t) \Big) \\
  & = dt \frac{1}{C_i} \Big(  g_{leak} (V_{rest} - V_i) + g_{syn, i}(t)(V_{syn} - V_i) + I_{stim}(t) + I_{noise, i}(t) \Big) \\ 
  & = dt \frac{1}{C_i} \Big(  g_{leak} (V_{rest} - V_i) + I_{noise, i}(t) + g_{syn, i}(t)(V_{syn} - V_i) + 0 \Big) \,\,\,\because I_{stim}(t) = 0 \,\forall t \\ 
  & = dt \frac{1}{C_i} \Big(  g_{leak} (V_{rest} - V_i) + I_{noise, i}(t) + g_{exc, i}(t)(V_{exc} - V_i) + g_{inh, i}(t)(V_{inh} - V_i) \Big) \,\,\, \because \text{it can be decomposed to excitatory and inhibitory spiking components} \\
  & = dt \frac{1}{C_i} \Big(  \underbrace{g_{leak} (V_{rest} - V_i)}_{A} + \underbrace{g_{noise, i}(t)(V_{noise} - V_i)}_{B} + \underbrace{g_{exc, i}(t)(V_{exc} - V_i)}_{C} + \underbrace{g_{inh, i}(t)(V_{inh} - V_i)}_{D} \Big) \\
\\
& \text{Now the formula is in a similar form as the code implementation.} \\
\end{align*}
$$


### Code implementation:
```python
## Line 191 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
v_Offset=ne.evaluate('1/tau*((Vrest-v) +(Vnoise-v)*Snoise+(Vexc-v)*Sexc+(Vinh-v)*Sinh)')

## Line 211 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
v[neuronsNotAtRefractory]=v[neuronsNotAtRefractory]+dt*v_Offset[neuronsNotAtRefractory]
```

``` python
## For clarify, they can be rewritten the following ways.
################################################################################
v_Offset=ne.evaluate(
  '1/tau*( (Vrest-v) + \
           (Vnoise-v)*Snoise + \
           (Vexc-v)*Sexc + \
           (Vinh-v)*Sinh \
          )' 
)

v[neuronsNotAtRefractory]=(
  v[neuronsNotAtRefractory] + 
  dt*v_Offset[neuronsNotAtRefractory]
)
```

### Variable `tau`: 
```python
## Line 407-409 in 'functions_sim.py` ##
################################################################################
tau=np.random.normal( system_parameters['tauSTN'] , system_parameters['tauSTN']*system_parameters['sigmaP'], N )
# and STN neurons
tau[N_STN:]=np.random.normal( system_parameters['tauGPe'], system_parameters['tauGPe']*system_parameters['sigmaP'], N_GPe )
```

```python
## Line 28, 42-45 in `functions_pars.py` ##
################################################################################
system_parameters.update({'sigmaP': 0.05 })

# set timescale for STN neurons
system_parameters.update({'tauSTN': 150})  # ms  
# and STN neurons
system_parameters.update({'tauGPe': 30.})  # ms 

```

-  `tau` is an ndarray of length `N`, where `N` is the total number of neurons: 1000 + 2.
- The first 1000 entries of the ndarray `tau` are instances of the random variable from `Norm(mean=tauSTN, std=tauSTN*sigmaP) = Norm(150, 150*0.05) = Norm(150, 7.5)`.
- The last 2 entries of the ndarray `tau` are instances of the random variable from `Norm(mean=tauGPe, std=tauGPe*sigmaP) = Norm(30, 30*0.05) = Norm(30, 1.5)`.
- By the forms of the equation from the paper and the code implementation, `tau` is equivalent to the $C_i$ (membrane capacitance) in the equation.
- Values: 
  - Paper: 
    - $C_i \stackrel{iid}{\sim} Norm(\mu_{C_i} = 3 \frac{\mu F}{cm^2}, \,\, \sigma = (0.05 * \mu_{C_i}) \frac{\mu F}{cm^2}) = Norm(3\frac{\mu F}{cm^2}, \,\, 0.15\frac{\mu F}{cm^2})$
    - In the original Fortran code and other paper that we started replicating, only STN population was modeled.
  - Code: 
    - For STN (first 1000 entries): $tau \sim Norm(150, \,\, 7.5)$
    - For GPe (last 2 entries): $tau \sim Norm(30, \,\,\, 1.5)$
- Scaling:
  - So `tau` or $C_i$ has been scaled by a factor of 50x:
  - `3 * 50 = 150`, `0.15 * 50 = 7.5`
- Questions: 
  - **??? Are the conductance for each of the components thereafter (A, B, C, D) also scaled by 50x to offset `tau`'s scaling? If so, does this method of decreasing the simulation computation time realistic?**

### Variable `Sleak` (or `g_leak`):
- Siemens is the unit for conductance, hence the author denotes conductance with `S` in the code. 
- In the papers, conductance is denoted with `g`, hence "leaky conductance" is referred to as `g_leak`.
- However, the variable is missing in the code because it is implicitly assumed to be `1 mS/cm^2`.
- Values: 
    - Paper: 
        - `g_leak = 0.02 mS/cm^2`
    - Code:
        - Implicitly `1 mS/cm^2` because it is factored out in the code.
- Scaling: 
    - As seen, the value used by the code is also a 50x scale of the value of the paper (`0.02 * 50 = 1`).
    - Thus, in addition to `tau` being scaled by 50x, `g_leak` is also scaled by 50x to 1 and factored out of the code implementation of dynamic function 5.
- Questions: 
    - In component A, the scaling of both `tau` and `g_leak` by a factor of 50x would cancel each other out mathematically, thus $(V_{rest} - V_i)$ is not scaled.
    - **??? Do we see similar cancelling out in component B, C, and D?**

### Variable `Vrest`:
- `Vrest` is an ndarray of length `N` where each of the value is VRestSTN = -38.0 mV.

### Varaible `Vnoise`: 
- `Vnoise` is an ndarray of length `N` where the first N_STN=1000 values are Vexc=0. mV and the last 2 values are Vinh=-80. mV.

### Variable `Vexc`:
- `Vexc` is a float equal to 0. mV.

### Variable `Vinh`:
- `Vinh` is a float equal to -80. mV.

### Variable `Snoise`:

### Variable `Sexc`:

### Variable `Sinh`:

## Equation 6: 
![Alt text](image-1.png)

## Equation 7: 
![Alt text](image-2.png)

$$
\begin{align*}
\tau_{syn} \frac{d g_{syn, i}}{d t} & = -g_{syn, i} + \kappa \frac{\tau_{syn}}{N} \sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a), \,\,\, \text{where } G_i := \{\text{All presynaptic neurons connected to neuron $i$}\} \\
\\
\implies dg_{syn, i} &= \frac{dt}{\tau_{syn}} \left[ -g_{syn, i} + \kappa \frac{\tau_{syn}}{N} \sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a) \right] \\
    & = \left[ \frac{dt}{\tau_{syn}} \cdot -g_{syn, i} \right] + \left[ \frac{dt}{\tau_{syn}} \kappa \frac{\tau_{syn}}{N} \sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a) \right] \\
    & = dt \cdot -\left[ \frac{g_{syn, i}}{\tau_{syn}} \right] + \left[ \frac{dt}{\tau_{syn}} \kappa \frac{\tau_{syn}}{N} \sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a) \right] \\
    & = dt \cdot -\left[ \frac{g_{syn, i}}{\tau_{syn}} \right] + dt \cdot \left[ \frac{\kappa}{\tau_{syn}} \frac{\tau_{syn}}{N} \sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a) \right] \\
    & = dt \cdot  \underbrace{- \left[ \frac{g_{syn, i}}{\tau_{syn}} \right] }_{A} + dt \cdot \left[ \underbrace{\frac{1}{N}}_{B} \underbrace{\frac{\kappa}{\tau_{syn}}}_{C} \underbrace{\tau_{syn}}_{D} \underbrace{\sum_{j \in G_i} w_{j \rightarrow i}(t) \sum_{l^j} \delta (t - t^j_{l^j} - t_a)}_{E} \right] \\
\end{align*}
$$

## Code implementation ( of (`Sexc` and `Sinh`) OR $g_{syn, i}$ ) 

```python
## Line 46 in `get_stationary_states_arange_x_homogeneousNetwork.py`
## (Anthony) - Initialize various system parameters via the function call 
################################################################################
STDPon, N, N_STN, VR, tauVT, VTspike, VTRest, tau_spike, V_spike, Vexc, Vinh, tauNoise, noiseIntensities, synaptic_offset_after_spike, tauSyn, dt, Tmax, t, recordedSpikes, kSave, StepsTauSynDelaySTNSTN, StepsTauSynDelayGPeGPe, StepsTauSynDelayGPeSTN, StepsTauSynDelaySTNGPe, a_delaySteps, b_delaySteps, c_delaySteps, delayedSpikingNeurons, numberOfDelayedTimeSteps , est_Spike_Times, switchOffSpikeTimes, basicFilterSpikingNeurons, lastSpikeTimeStep, evenPriorLastSpikeTimeStep, kWeightOutput, NeuronIndices, start_time, borderForScaningForSpikeValues, borderForScaningNeuronsPassingSpikingThreshold, STDPAlreadyOn = functions_sim.initialize_system( system_parameters )

## Line 51-52 in `get_stationary_states_arange_x_homogeneousNetwork.py`
## (Anthony) - Initialize various parameters for the nodes
################################################################################
# apply initial conditions
Vrest , tau , v, s, Snoise, VT, Vnoise =  functions_sim.set_initial_conditions_for_nodes( system_parameters )

## Line 182-185 in `get_stationary_states_arange_x_homogeneousNetwork.py`
## (Anthony) - Extract the conductance for excitatory synapses and inhibitory synapses
################################################################################
# first column of s contains values for exc. conductances
# second column values for inhibitory conducances
Sexc=s[:,0]
Sinh=s[:,1]
```
- This section shows two different function calls to initialize the system and nodes' parameters. Respectively, `functions_sim.initialize_system()` and `functions_sim.set_initial_conditions_for_nodes()`.
- `s` is returned from the `functions_sim.set_initial_conditions_for_nodes()` call, then in line 182-185, the Nx2 ndarray array is split by excitatory and inhibitory synapses for convenience later. 
- `Sexc` $\equiv$ "Excitatory synapse Conductance" --> This variable is updated by a dynamic function, so this ndarray is used to hold the values.
- `Sinh` $\equiv$ "Inhibitory synapse Conducatance" --> This variable is updated by a dynamic function, so this ndarray is used to hold the values. 



``` python
## Line 473-476 in `functions_sim.py`
## (Anthony) - Get values from dictionary: tauSynExc = 1.0, tauSynInh = 3.3, tauNoise = 1.0 millisecond
################################################################################
# synaptic time scales
tauSynExc=system_parameters['tauSynExc'] # synaptic time constant [ms] according to Ebert et al. 2014
tauSynInh=system_parameters['tauSynInh']
tauNoise = system_parameters['tauNoise']

## Line 483-485 in `functions_sim.py`
################################################################################
# preevaluate synaptic quantities to speed up simulation
synaptic_offset_after_spike=1./(float(N)) # mV
tauSyn=np.array([1/tauSynExc,1/tauSynInh])

## Line 190-193 in `get_stationary_states_arange_x_homogeneousNetwork.py`
## (Anthony) - `s_Offset` is component A of the reordered equation 7 above.
################################################################################
# calculate right-hand site of equation for membrane potential
v_Offset=ne.evaluate('1/tau*((Vrest-v) +(Vnoise-v)*Snoise+(Vexc-v)*Sexc+(Vinh-v)*Sinh)')
# synaptic conductances
s_Offset=-np.multiply(s,tauSyn)

## Line 212 in `get_stationary_states_arange_x_homogeneousNetworki.py`
## (Anthony) - Partial update of the conductance at current Euler step.
################################################################################
s=ne.evaluate('s+dt*s_Offset')
```
- The values are read in from the Python dictionary `system_parameters`, of which has the following key-value pairs.
    - `tauSynExc` = 1.0 ms
    - `tauSynInh` = 3.3 ms
    - `tauNoise` = 1.0 ms
- `synaptic_offset_after_spike`: is equivalent to component B of the reordered equation 7 above. 
- `tauSyn`: 
    - An ndarray of length 2, holding the inverse time-constant for excitatory and inhibitory synapses respectively.
    - This is equivalent to the denominator of component C of the reordered equation 7 above.
- `s_Offset`: 
    - Equivalent to component A of the reordered equation 7 above.
- `s`: 
    - The synaptic conductance `s` is partially updated by multiplying component A by time-step `dt` and added to the value of `s` from the previous time-step.
    - Component B, C, (D), E are part of the second part of the synaptic conductance update. (Component D is is parenthesized because it seems to be missing from the code implementation.)
- kappa
    - It scales the maximal coupling strengh, $\kappa = 8 \, mS/cm^2$ (in the paper), however, the code set this value as `400`.
    - The scaling up of 


``` python
## Line 339-340 in `get_stationary_states_arange_x_homogeneousNetworki.py`
## (Anthony) - Later matrix multiplication would sum all the weights of spiked presynaptic partners of each neuron.
################################################################################
# spikes are arriving and cause upades of synaptic conductances
activeConnectionWeights=scipy.sparse.csc_matrix( ( np.array(cMatrix[synapsesOfArrivingSpikes[1:,1],synapsesOfArrivingSpikes[1:,0]])[0], (synapsesOfArrivingSpikes[1:,1],synapsesOfArrivingSpikes[1:,0]) ), shape=(N,N) )

## Line 29-34 in `functions_pars.py`
## (Anthony) - This is equivalent to what the paper terms "maximal coupling strength (kappa)", which the paper initialized at 8 mS/cm^2 for excitatory STN neurons.
################################################################################
# exc coupling
system_parameters.update({'cMaxExc': 400})
system_parameters.update({'cExcInit': 200.0})
# inh coupling
system_parameters.update({'cMaxInh': 0.0})
system_parameters.update({'cInhInit': 0.0})

## Line 660-663 in `functions_sim.py`
################################################################################
# synaptic time scales
tauSynExc=system_parameters['tauSynExc']  # ms # synaptic time constant [ms] according to Ebert et al. 2014
tauSynInh=system_parameters['tauSynInh']  # ms
tauNoise = system_parameters['tauNoise']  # ms

## Line 798-799 in `get_stationary_states_arange_x_homogeneousNetworki.py`
## (Anthony) - Placeholder that will be updated soon.
################################################################################
# create filter to read out exc/inh neurons
basicFilterSpikingNeurons=np.zeros( (N,2) )

## Line 806-810 in `functions_sim.py`
## (Anthony) - Update the placeholder; equivalent to component C of the reordered equation 7.
## (Anthony) - Ignore the `np.ones()` as it is just to initialize an ndarray; it is a multiplicative identity anyways.
################################################################################
# filter already consideres coupling strengths and synaptic timescales according to differential equations
# excitatory weights
basicFilterSpikingNeurons[:N_STN,0]=cMaxExc/tauSynExc*np.ones(N_STN)
# inhibitory weights
basicFilterSpikingNeurons[N_STN:,1]=cMaxInh/tauSynInh*np.ones(N_GPe)
```
- `activeConnectionWeights` is initialized as a sparse matrix because the weight matrix (`cMatrix`) is sparse. 
    - `cMatrix` is an ndarray 2D array storing the weight of all pairwise connections - If there isn't a connection, the weight would be 0.
- `basicFilterSpikingNeurons` is a 2D ndarray with excitatory synapses in the first column and the inhibitory neuron in the second column.
    - The value recorded here is equivalent to component C of the reordered equation 7 above.

``` python
## Line 342-346 in `get_stationary_states_arange_x_homogeneousNetworki.py`
## (Anthony): 
##   - `synaptic_offset_after_spike` = 1/N = Component B of the reordered equation 7.
##   - `activeConnectionWeights` = Component E of the reordered equation 7.
##   - `basicFilterSpikingNeurons` = Component C of the reordered equation 7.
##   - This addes the second part of the dg update to the g (equivalent to the `s` in the code).
##   - HOWEVER!!! --> Component D is mising from the code implementation??? This is problematic for the conductance updates of the inhibitory synapse because `tauSynInh` is not 1.
################################################################################
# update synaptic weight
# this matrix product effectively calculates the sum over weights of active synapses 
# this sum is then added to 's' while rescaling with 'synaptic_offset_after_spike' = 1/N
# first column of s are excitatory and second column are inhibitory synapses
s+=synaptic_offset_after_spike*activeConnectionWeights.dot( basicFilterSpikingNeurons )
```
- Here we combine all components together. 
    - `synaptic_offset_after_spike` = Component B
    - `activeConnectionWeights` = Component E
    - `basicFilterSpikingNeurons` = Component C
- Conclusion: 
    - Component D is missing in the code, however, when `tau_syn` = 1, it does not matter as it would be a multiplicative identity.
    - **TODO: ??? Ask Justus why is component D missing from the code here? Doesn't this impact the dynamic of how the synaptic conductance for excitatory and inhibitory synapses update at each Euler step?**



## Equation 8:
![Alt text](image-3.png)

## Equation 9: 
![Alt text](image-4.png)

$$
\begin{align*}
\tau_{syn} \frac{d g_{noise, i}}{d t} & = -g_{noise, i} + \kappa_{noise} \tau_{syn} \sum_{k_i} \delta (t_{k_i} - t) \\
\\
\implies d g_{noise, i} & = \left( \frac{d t}{\tau_{syn}} \cdot -g_{noise, i} \right) + \left( \frac{d t}{\tau_{syn}} \cdot \kappa_{noise} \tau_{syn} \sum_{k_i}\delta (t_{k_i} - t) \right) \\
    & = dt \left[ \left( \frac{1}{\tau_{syn}} \cdot -g_{noise, i} \right) + \left( \frac{1}{\tau_{syn}} \cdot \kappa_{noise} \tau_{syn} \sum_{k_i}\delta (t_{k_i} - t) \right) \right] \\
    & = dt \left[ \left( \frac{-g_{noise, i}}{\tau_{syn}} \right) + \left( \frac{1}{\tau_{syn}} \cdot \kappa_{noise} \tau_{syn} \sum_{k_i}\delta (t_{k_i} - t) \right) \right] \\
    & = dt \left[ \left( \underbrace{\frac{-g_{noise, i}}{\tau_{syn}}}_{A} \right) + \left( \underbrace{\kappa_{noise}}_{B} \sum_{k_i} \underbrace{\delta (t_{k_i} - t)}_{C} \right) \right] \\
    & = dt \left[ \left( \underbrace{\frac{-g_{noise, i}}{\tau_{syn}}}_{\tau_{syn} = 1 ms} \right) + \left( \underbrace{\kappa_{noise}}_{=0.026 \frac{mS}{cm^2}} \sum_{k_i} \underbrace{\delta (t_{k_i} - t)}_{\text{Dirac Delta Function}} \right) \right] \\
\end{align*}
$$

### Code implementation: 

```python
## Line 191 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
v_Offset=ne.evaluate('1/tau*((Vrest-v) +(Vnoise-v)*Snoise+(Vexc-v)*Sexc+(Vinh-v)*Sinh)')

## Line 195 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
Snoise_Offset=-Snoise/tauNoise  # tauNoise is set as a constant = 1.0 millisecond

## Line 213 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
Snoise=ne.evaluate('Snoise+dt*Snoise_Offset')

## Line 216-218 in `get_stationary_states_arange_x_homogeneousNetwork.py'
################################################################################
# update noise conductances if Poisson spikes arrive
# spikeArrival[ currentSlotInSpikeArrival ] cotains the number of arriving spikes, i.e. 0, 1, ...
Snoise+=noiseIntensities/tauNoise*spikeArrival[ currentSlotInSpikeArrival ]
```

- `tauNoise` is denoted as $\tau_{syn}$ in the paper, and both parameters are set to 1.0 millisecond.
- Line 195 aligns with A of the reordered equation.
- Line 218: 
    - `noiseIntensities`:
        - The code sets this value to be `1.3`.
        - $\kappa_{noise} = 0.026 \frac{mS}{cm^2}$ in B corresponds to this variable. 
        - The code's value is scaled by a factor of 50x from the value stated by the paper: `0.026 * 50 = 1.3`
    - `tauNoise`:
        - As seen in the derivation of equation 9 from the paper, $\tau_{syn}$ has been cancelled out and thus I believe should NOT be included in the code line 218.
        - `tauNoise = 1`, even though I believe is mathematically incorrect, the results should be okay due to 1 being a multiplicative identity. HOWEVER, if the value were to ever be changed via the `functions_pars.py` to any other value (suppose we are trying to model something with slower synaptic time-constant), then this simulation would not be correct.
        - TODO: Discuss and ask the author if this should be fixed.
    - `spikeArrival`: 
        - The `spikeArrival` ndarray records the number of spiked presynaptic partners of each neuron.
        - It models the Dirac Delta Function (or C) of the re-organized equation 9 above.



```python
## Line 479-480 in 'functions_sim.py`
################################################################################
noiseIntensities=np.full( N, system_parameters['noiseSTN'] )
noiseIntensities[N_STN:]=np.full( N_GPe, system_parameters['noiseGPe'] )

## Line 61-62 in 'functions_pars.py`
################################################################################
system_parameters.update({'noiseSTN': 1.3})
system_parameters.update({'noiseGPe': 2.0})
```
- This shows how `noiseIntensities` is created. It is just an ndarray of size `N = N_STN + N_GPe = 1000 + 2`.
- The first `N_STN=1000` elements of this ndarray has the value `noiseSTN = 1.3`.
- The last 2 elements of this ndarray has the value `noiseGPe = 2.0`
- ??? Does this mean that if we were to mode an inhibitory neuron and want to scale by 1/50, then the actual intensity would be `0.04`?  


### Conclusion: 
- We see that `tauNoise` is incorrectly included in line 218, however, because the variable is set as a constant 1, it does not create issue with the simulation.
- We see that $\kappa_{noise}$ is being modeled as conductance and scaled by a factor of 50x.

### 

## Parameter values: 
![Alt text](image-5.png)